In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [19]:
#week_day_Oct_list = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]
week_day_Oct_list = [2, 9, 16, 23, 30]

In [20]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in week_day_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['MD_flow_minute'][minute_idx])

In [21]:
len(link_day_minute_Oct_list)

14400

In [23]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 600)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [24]:
np.size(x, 0), np.size(x, 1)

(24, 600)

In [25]:
x[:,:2]

matrix([[ 2388.85023266,  2317.99697662],
        [ 2235.15683479,  2298.51147847],
        [ 2863.08941813,  2848.89387585],
        [ 2894.26108129,  2881.38692522],
        [ 2765.61848262,  2812.96945915],
        [ 2763.5216542 ,  2607.71742863],
        [ 2364.75834274,  2321.03239065],
        [ 2369.3219381 ,  2357.92762384],
        [ 3229.29411564,  3219.53929543],
        [ 3197.48909879,  3066.32983642],
        [ 2656.27198223,  2663.43977835],
        [ 2414.96150877,  2201.83162751],
        [ 2244.96803408,  2307.72951123],
        [ 2491.62179065,  2428.84971598],
        [ 2410.08382257,  2446.05903492],
        [ 1959.05874575,  1931.72972169],
        [ 2356.53682403,  2375.54157664],
        [ 2376.87214354,  2343.89526826],
        [ 2608.89905719,  2652.90711499],
        [ 2661.4108332 ,  2551.98646275],
        [ 2740.06180662,  2860.73631523],
        [ 2651.29507733,  2610.22571774],
        [ 2298.6409424 ,  2367.80269209],
        [ 2330.89870134,  2336.299

In [26]:
np.size(A,0), np.size(A,1)

(24, 140)

In [27]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [7e+00, 4e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 26
 AA' NZ     : 3.250e+02
 Factor NZ  : 3.510e+02
 Factor Ops : 6.201e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -6.53255011e+07  0.00000000e+00  8.07e+03 1.82e+03  1.00e+06     0s
   1  -3.03520010e+07 -1.80286186e+07  1.28e+03 2.90e+02  2.26e+05     0s
   2  -2.13405916e+07 -2.22263939e+07  5.00e+01 1.13e+01  3.25e+04     0s
   3  -2.11630780e+07 -2.13392478e+07  5.00e-05 1.13e-05  3.1

In [28]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekday_MD.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [29]:
lam_list

[9.600048532385772e-12,
 3013.482564048393,
 1.053768987577737e-11,
 579.2524228380736,
 0.0007007531874017973,
 1300.0588171509787,
 3.1799479668009433e-09,
 0.00018406057456346776,
 0.003275025817570082,
 2595.7689246226823,
 96.97435381921144,
 0.0004632217257994585,
 0.0032928557800396413,
 0.003284558523909978,
 2867.8839837819355,
 0.24642510174544618,
 0.0009822882303804695,
 0.0009829744117517032,
 1153.9236229672028,
 169.2010454328214,
 2429.264405223341,
 1352.441744702072,
 1276.0077138052102,
 6.500297576483047e-05,
 2119.5814497438946,
 0.004593993522409953,
 0.001996472022808414,
 2343.3322532493803,
 626.5919903070039,
 0.1179282174742351,
 0.13290129390697958,
 2333.8217580480537,
 2037.9399908954356,
 0.0019728156640467787,
 0.0019642578919099735,
 1.3867311148602524e-05,
 1.386725881208882e-05,
 1.3866725120439624e-05,
 3.207890576299974,
 1563.5200907209728,
 1035.0284971243996,
 0.10986127894491915,
 0.8614805878293019,
 0.2779362831365799,
 2274.790056320499,
 0.2